# Name: Lara Mechling
## Title: 2022 Fall Shopify Data Science Internship Challenge
### Date: May 18th, 2022

#### 1. On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis.

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [3]:
# import csv file
df = pd.read_csv('dataset.csv')
df.sample(5)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
4106,4107,26,878,176,1,debit,2017-03-20 4:32:18
869,870,89,847,392,2,cash,2017-03-14 21:16:20
2056,2057,35,706,164,1,credit_card,2017-03-01 8:34:45
2819,2820,97,799,486,3,credit_card,2017-03-23 0:39:05
318,319,83,951,258,2,debit,2017-03-04 5:10:36


Average order value was calculated incorrectly at $3145.13. The first step is to determine how this number was calculated.

In [6]:
order_sum = sum(df['order_amount'])
order_total = len(df.index)
aov = order_sum / order_total
aov

3145.128

What could be driving the order price so high? 

In [15]:
df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


It looks like the max order value of $704,000 is driving the average order value up. Let's take a look at the entry('s) with that order value.

In [18]:
max = df.query('order_amount == 704000')
max

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00
1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00
1436,1437,42,607,704000,2000,credit_card,2017-03-11 4:00:00
1562,1563,42,607,704000,2000,credit_card,2017-03-19 4:00:00
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00
2153,2154,42,607,704000,2000,credit_card,2017-03-12 4:00:00
2297,2298,42,607,704000,2000,credit_card,2017-03-07 4:00:00


In [19]:
max.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 15 to 4882
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        17 non-null     int64 
 1   shop_id         17 non-null     int64 
 2   user_id         17 non-null     int64 
 3   order_amount    17 non-null     int64 
 4   total_items     17 non-null     int64 
 5   payment_method  17 non-null     object
 6   created_at      17 non-null     object
dtypes: int64(5), object(2)
memory usage: 1.1+ KB


Shop 42 has 17 orders at $704,000 per order effectively driving the AOV very high in comparison to the other order amounts. 

Since each shop sells a single type of item, and the item values cary from shop to shop I would look at the average order value for each shop.

In [31]:
# group dataframe by shop

df['Occur'] = df.groupby('shop_id')['shop_id'].transform('size')
df_group = df.groupby(['shop_id']).agg({'order_amount':'sum', 'Occur':'first'}).reset_index()
df_group

,shop_id,order_amount,Occur
0,1,13588,44
1,2,9588,55
2,3,14652,48
3,4,13184,51
4,5,13064,45
...,...,...,...
95,96,16830,51
96,97,15552,48
97,98,14231,58
98,99,18330,54


In [32]:
# find aov by shop

df_group['shop_aov'] = df_group['order_amount']/df_group['Occur']
df_group

,shop_id,order_amount,Occur,shop_aov
0,1,13588,44,308.818182
1,2,9588,55,174.327273
2,3,14652,48,305.250000
3,4,13184,51,258.509804
4,5,13064,45,290.311111
...,...,...,...,...
95,96,16830,51,330.000000
96,97,15552,48,324.000000
97,98,14231,58,245.362069
98,99,18330,54,339.444444


Here we can see the average order value for each indivifual shop. Lets look at the metrics.

In [33]:
# look at metrics

df_group.describe()

,shop_id,order_amount,Occur,shop_aov
count,100.000000,1.000000e+02,100.000000,100.000000
mean,50.500000,1.572564e+05,50.000000,3136.834088
std,29.011492,1.216218e+06,7.287737,23935.881130
min,1.000000,6.840000e+03,35.000000,162.857143
25%,25.750000,1.293050e+04,44.750000,263.675962
50%,50.500000,1.488750e+04,50.000000,308.889758
75%,75.250000,1.760000e+04,55.000000,336.628352
max,100.000000,1.199018e+07,68.000000,235101.490196


#### Response

Looking at all of the data for the shop sales I conclude that the AOV is only an appropriate metric if you are looking at it on a shop by shop basis. Looking at the overall AOV is not telling as each shop sells goods at different values and the number is highly skewed by large numbers in the dataset. A better metric to look at to get an idea of where the majority of sales lie will be the MEDIAN order value. Looking at the entire dataset the median order value is: $284. This is much more representative of the shop sales as a whole and does not allow for the skewed average calculation due to the shop with high priced items. 

#### 2.a How many orders were shipped by Speedy Express in total?

SELECT COUNT(ShipperID) FROM Orders AS ShippedBySpeedyExpress
WHERE ShipperID = 1;

#### Result

54

#### 2.b What is the last name of the employee with the most orders?

SELECT EmployeeID, COUNT(*) 
FROM Orders 
GROUP BY EmployeeID;

#### Result

Peacock

#### 2.c What product was ordered the most by customers in Germany?

This is my first time using SQL and unfortunately I do not know the best route to get this answer. I know the general idea would be to query the information from the orders, products, and customer tables, then count the times the products were ordered by cutomers residing in Germany.